# **Setup**

In [3]:
# Auto-reload edited scripts
%load_ext autoreload
%autoreload 2

# Logging
from ngram_prep.ngram_acquire.logger import setup_logger

# Ngram pivot functions
from pathlib import Path
from ngram_prep.ngram_pivot.config import PipelineConfig
from ngram_prep.ngram_pivot.pipeline import run_pivot_pipeline

# Utilities
from ngram_prep.utilities.peek import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Log to file

In [4]:
setup_logger(
    db_path="/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams_pivoted.db",
    console=False,
    rotate=True,
    max_bytes=100_000_000,
    backup_count=5,
    force=True
)

OSError: [Errno 30] Read-only file system: '/vast'

# **Pivot Multigrams to Create Yearly Indices**

In [ ]:
pipeline_cfg = PipelineConfig(
    src_db=Path("/scratch/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/1grams_processed.db"),
    dst_db=Path("/scratch/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/1grams_pivoted.db"),
    tmp_dir=Path("/scratch/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/pivot_tmp"),
    num_workers=40,
    num_initial_work_units=80,
    max_split_depth=50,
    work_unit_claim_order="sequential",
    split_check_interval_s=45.0,
    progress_every_s=15.0,
    mode="restart",
    ingest_batch_items=1_00_000,
    max_items_per_bucket=10_000_000,
    max_bytes_per_bucket=256 * 1024 * 1024
)

run_pivot_pipeline(pipeline_cfg)


PARALLEL N-GRAM DATABASE PIVOT
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Start Time: 2025-10-24 16:55:33
Mode:       RESTART

Configuration
════════════════════════════════════════════════════════════════════════════════════════════════════
Source DB:            ...dk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/1grams_processed.db
Target DB:            .../edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/1grams_pivoted.db
Temp directory:       /scratch/edk202/NLP_corpora/Google_Books/20200217/eng/1gram_files/pivot_tmp

Parallelism
────────────────────────────────────────────────────────────────────────────────────────────────────
Workers:              40
Initial work units:   80
Max split depth:      50
Split check interval: 45.0s

Database Profiles
────────────────────────────────────────────────────────────────────────────────────────────────────
Reader profile:       read:packed24
Writer profile:       writ

# **Inspect the Procesed Database**
## `db_head`: Print the first _N_ key–value pairs

In [ ]:
db_path = "/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams_processed.db"

db_head(db_path, key_format="utf-8", value_format="packed", n=5)

## `db_peek`: Print _N_ key-value pairs starting at the specified key

In [ ]:
db_path = "/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams_processed.db"

db_peek(db_path, start_key=b"quick brown <UNK> <UNK> <UNK>", key_format="utf-8", value_format="packed", n=5)


## `db_peek_prefix`: Print key-value pairs containing the specified prefix

In [ ]:
db_path = "/vast/edk202/NLP_corpora/Google_Books/20200217/eng/5gram_files/5grams_processed.db"

db_peek_prefix(db_path, prefix=b"<UNK> united state <UNK> <UNK>", key_format="utf-8", value_format="summary", n=1)